# VAE Training

## imports

In [ ]:
import os, time
from glob import glob
import numpy as np

from model.VariationalAutoencoder import VariationalAutoencoder
from keras.preprocessing.image import ImageDataGenerator

from keras.preprocessing.image import ImageDataGenerator, load_img, save_img, img_to_array, array_to_img

# environment variables

In [4]:
MODEL_FOLDER = os.environ.get('HOME') + '/model3'
FILE_MODEL = 'weights.h5'

# VAE

## model variables

In [5]:
vae = VariationalAutoencoder()
vae.model.load_weights(os.path.join(MODEL_FOLDER, 'weights.h5'))  

In [13]:
from PIL import Image
from PIL.ImageShow import show

#image_path = '/Volumes/My Passport/PFM/output/training_png/Week9_39301_G11_4_Cells_34.png'
#image_path = '/Volumes/My Passport/PFM/output/training_png/Week5_28921_B02_2_Cells_13.png'
#image_path = '/Volumes/My Passport/PFM/output/training_png/Week5_28921_B02_3_Cells_39.png'
image_path = '/Volumes/My Passport/PFM/output/training_png/Week7_34641_B02_1_Cells_28.png'

img = Image.open(image_path)
array_img  = img_to_array(img) # keras

#array_img = array_img / 256.

#print(array_img)

img.show()

print('image format: ' + str(img.format))
#print('image shape: ' + str(img.shape))
print('image mode: ' + img.mode)
print('image size: ' + str(img.size))
print('image width: ' + str(img.width))
print('image height: ' + str(img.height))
print('image palette: ' + str(img.palette))
print('image info: ' + str(img.info))

print('image to array: ' + str(array_img.shape))

array_img = np.expand_dims(array_img, axis=0)


vae.encoder.summary()

print('image to array after expand: ' + str(array_img.shape))

z_result = vae.encoder.predict(array_img)
reco = vae.decoder.predict(z_result).squeeze()

#print(reco)

print(reco.shape)

# scaled
outimg = array_to_img(reco, scale=True)

show(outimg)

print(reco)


image format: PNG
image mode: RGB
image size: (128, 128)
image width: 128
image height: 128
image palette: None
image info: {}
image to array: (128, 128, 3)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 64, 64, 32)   0           encoder_conv_1[0][0]             
__________________________________________________________________________________________________
encoder_conv_2 (Conv2D)         (N

(128, 128, 3)
[[[ 5.16063906e-03  1.84085779e-03 -4.66583483e-03]
  [ 2.19300017e-03 -4.47656028e-04 -1.12136081e-03]
  [ 1.09537132e-03 -4.47094906e-04 -1.95633620e-04]
  ...
  [ 2.91783363e-04 -7.32385088e-04 -2.66667455e-04]
  [ 8.20700079e-05 -9.48917121e-04 -1.65261328e-04]
  [ 1.01856887e-03 -7.86175951e-04 -1.12435967e-03]]

 [[ 1.37223303e-03  1.36149349e-03 -1.07144937e-03]
  [-9.67402011e-05 -3.87595268e-04  2.75148079e-03]
  [ 7.38462433e-04 -1.32904388e-05  2.56534666e-04]
  ...
  [ 9.64440405e-05 -7.18115130e-04  5.65864146e-04]
  [-4.19128686e-04 -1.21639902e-03 -3.31290066e-04]
  [ 2.83636153e-04 -2.60745874e-04  1.58237293e-03]]

 [[ 6.99073076e-04 -1.61364209e-04  9.32477415e-04]
  [ 1.95872039e-04  2.41106376e-04  8.21568072e-04]
  [ 2.20055506e-03 -1.47288665e-05 -2.12918967e-04]
  ...
  [ 6.64010644e-04 -9.28968657e-04 -1.14072114e-04]
  [ 7.73951411e-04 -9.62502323e-04 -6.12348318e-04]
  [-2.11082399e-04 -1.13568502e-03 -5.50303608e-04]]

 ...

 [[-6.77617267e-04 -